#### After mapping each AOI to 2 time windows for each car, we will predict the KPI of the AOI

There are in total 1226 AOIs for HH and 940 for VH.

We have X windows for each AOI based on the filter. 

The weights should be found based on the Two closest points, and the avg. speed that it has at these two points.

The logic is that there are always(*) a second on each side of the AOI. By using the distance and the speed, one can determine 

1 and 1,1

Total distance from 

or 1 and 2

(*) If the speed differs alot between the seconds they could both be on the same side, but that seems to be very unlikely. 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import h5py
from scipy.interpolate import CubicSpline
from copy import deepcopy
from tqdm import tqdm
import re
import csv
from pathlib import Path

In [8]:
trip = np.array(["1.0", "2.0", "3.0", "7.63"], dtype=object)

for idx, item in enumerate(trip):
    item = item.decode('utf-8')
    new_item = int(item)


AttributeError: 'str' object has no attribute 'decode'

In [4]:
x = np.array(["a", "b", "c", "7.63"], dtype=object)

# Replace the specific element with a float
x[3] = float(x[3])
x

array(['a', 'b', 'c', 7.63], dtype=object)

In [3]:
import h5py

def load_from_hdf5(filename):

    def unpack_group(group):
        unpacked_data = {}
        for key, item in group.items():
            if isinstance(item, h5py.Group):
                unpacked_data[key] = unpack_group(item)
            else:
                unpacked_data[key] = item[:].tolist()  # Convert dataset to list
        return unpacked_data

    with h5py.File(filename, 'r') as hdf_file:
        loaded_mapping = unpack_group(hdf_file)
    
    return loaded_mapping

# Takes about 40 seconds to load
h5py_mapping_hh = load_from_hdf5("../data/AOI/mapping_hh_time_to_location.hdf5")
h5py_mapping_vh = load_from_hdf5("../data/AOI/mapping_hh_time_to_location.hdf5")


In [8]:
def get_total_distance(data):
    first_values = []

    for trip_name, passes in data.items():
        for pass_name, segments in passes.items():
            for segment_name, values in segments.items():
                if segment_name in ['distance_segment_second_1', 'distance_segment_second_2']:
                    # Extract the first value and add to the list
                    first_values.append(values[0])

    total_distance = np.sum(first_values)
    return total_distance


def time_to_drive_X_meters(speed_kmh, distance_m=10):
    # Convert speed from km/h to m/s
    speed_ms = speed_kmh * (1000 / 3600)

    # Calculate time in seconds
    time_seconds = distance_m / speed_ms
    
    return time_seconds


# Weight functions 
def ln_of_ratio_sum_to_1(values):
    values_total = np.sum(values)
    weight_values = -np.log(values / values_total)
    total = np.sum(weight_values)
    weight_values_1 = weight_values / total
    return weight_values_1


def inverse_of_ratio_sum_to_1(values):
    values_total = np.sum(values)
    weight_values = (values / values_total)**(-1)
    total = np.sum(weight_values)
    weight_values_1 = weight_values / total
    return weight_values_1



In [33]:
distances_for_9 =[]
for key in h5py_mapping_hh["9"]:
    for trip in h5py_mapping_hh["9"][key]:
        for x in h5py_mapping_hh["9"][key][trip]:
            distances_for_9.append(h5py_mapping_hh["9"][key][trip][x][0])

print(distances_for_9)

[2.540231352824462e-05, 8.632562717940327e-05, 5.928268992285774e-05, 3.743649471121573e-05, 3.761562002457572e-05, 6.696434950771082e-05, 9.429546881253799e-05, 2.1146310142677474e-05, 8.801178253613132e-05, 3.906145347968156e-05, 5.4721685698429826e-05, 5.108344038156179e-05, 6.696555831254422e-05, 4.3119168125896057e-05, 5.046870808750601e-05, 1.4847958831495672e-05, 4.214348501620518e-05, 6.117891089454114e-05, 7.55634327034363e-05, 3.5240022653215575e-05, 6.374089683910665e-05, 3.79892879657307e-05, 5.76945331131384e-05, 4.8893849801930886e-05, 7.181647401909915e-05, 1.4249306321962006e-05, 3.871497916288721e-05, 3.449529934849564e-05, 4.20503636108622e-05, 1.1988063861835858e-05, 9.316923197065222e-05, 4.715698446513804e-05, 7.602746033591527e-05, 1.6457634408877064e-05, 4.124903722339637e-05, 1.4589642162581957e-05, 3.474366787991113e-05, 4.592517960614626e-09, 4.003571301966519e-05, 5.691958132192554e-05, 6.081587477808251e-05, 2.093077968852202e-05, 4.246165975362833e-05, 3.40

In [35]:
ln_weights = ln_of_ratio_sum_to_1(distances_for_9)
inv_weights = inverse_of_ratio_sum_to_1(distances_for_9)

# print the sorted weights
print(sorted(ln_weights))
print(sorted(inv_weights))

[0.010739603564782837, 0.010775854535641212, 0.010947662710957907, 0.011006023980421194, 0.011042158196934085, 0.011277487163175183, 0.01138927827833483, 0.011407748677380551, 0.0114165499069709, 0.011539821782041963, 0.01156118861561933, 0.011772183358522727, 0.011772237819171171, 0.011919570134219728, 0.011921078119953651, 0.012029499273688774, 0.012044846734058838, 0.012050673443883716, 0.012062802917620371, 0.012139837279223565, 0.012207448362752029, 0.012221763505341142, 0.012256371533610247, 0.012262562305534597, 0.012314536039642486, 0.012335639825906327, 0.012381369465790711, 0.012588937618441964, 0.012625463979491576, 0.012720531062548952, 0.012721108320036454, 0.012830231380947084, 0.012850577515929154, 0.012871620469845961, 0.013100295923478906, 0.013146655294934355, 0.013169347419500571, 0.013176021224707688, 0.013234068943069879, 0.013284006592812696, 0.013324143935390161, 0.013398469787270764, 0.013425349838397304, 0.013482438449653933, 0.0135122609173096, 0.0135266621293

In [37]:
test = np.array([2.54e-05, 5.93e-05, 8.63e-05])

print(ln_of_ratio_sum_to_1(test))
print(inverse_of_ratio_sum_to_1(test))



[0.52247055 0.29016754 0.1873619 ]
[0.58050008 0.2486459  0.17085402]


In [12]:
for car in h5py_mapping_hh["0"]:
    print(h5py_mapping_hh["0"][car])
    for trip in h5py_mapping_hh["0"][car]:
        print(h5py_mapping_hh["0"][car][trip]) 
        for distance_segment_second in h5py_mapping_hh["0"][car][trip]:
            print(h5py_mapping_hh["0"][car][trip][distance_segment_second])
            break
        break
    break


{'pass_1': {'distance_segment_second_1': [2.3317746094376914e-05, 0.0, 0.0], 'distance_segment_second_2': [3.658563558098666e-05, 0.0, 1.0]}, 'pass_11': {'distance_segment_second_1': [6.483034333046726e-05, 2.0, 2.0], 'distance_segment_second_2': [9.578165683427985e-05, 2.0, 1.0]}, 'pass_13': {'distance_segment_second_1': [3.2137861824301134e-05, 3.0, 1.0], 'distance_segment_second_2': [4.7965809663709314e-05, 3.0, 0.0]}, 'pass_15': {'distance_segment_second_1': [7.158024588141076e-05, 5.0, 0.0], 'distance_segment_second_2': [8.890653162778374e-05, 5.0, 1.0]}, 'pass_17': {'distance_segment_second_1': [2.3883416921457747e-05, 7.0, 1.0], 'distance_segment_second_2': [6.0817756440021506e-05, 7.0, 0.0]}, 'pass_3': {'distance_segment_second_1': [4.2139430466692684e-05, 10.0, 0.0], 'distance_segment_second_2': [6.319870650188742e-05, 10.0, 1.0]}, 'pass_5': {'distance_segment_second_1': [9.47361156045296e-05, 12.0, 0.0], 'distance_segment_second_2': [9.47361156045296e-05, 12.0, 1.0]}, 'pass_9

In [13]:
mapping_with_weights_hh = h5py_mapping_hh
mapping_with_weights_vh = h5py_mapping_vh
maps = [mapping_with_weights_hh, mapping_with_weights_vh]

new_mapping = {}

for mapping in maps:
    indexes_strings = list(mapping.keys())
    indexes = np.sort([int(x) for x in mapping])
    for index in indexes:
        new_mapping[index] = []
        distances = []
        segments = []
        seconds = []

        # total_distance_for_index = get_total_distance(mapping[str(index)])
        # Now go into each individual trip and segment and calculate the weight
        for car in mapping[str(index)]:
            for trip in mapping[str(index)][car]:
                for distance_segment_second in mapping[str(index)][car][trip]:
                    distances.append(mapping[str(index)][car][trip][distance_segment_second][0])
                    segments.append(mapping[str(index)][car][trip][distance_segment_second][1])
                    seconds.append(mapping[str(index)][car][trip][distance_segment_second][2])

        
        weight_ln_ratio = ln_of_ratio_sum_to_1(distances)
        weight_inverse_ratio = inverse_of_ratio_sum_to_1(distances)

        for i in range(len(distances)):
            new_mapping[index].append([distances[i], segments[i], seconds[i], weight_ln_ratio[i], weight_inverse_ratio[i]])


In [14]:
new_mapping[index]

[[8.508740212138433e-05, 4.0, 49.0, 0.06547794769130515, 0.06307755338794452],
 [9.232527915058013e-05, 4.0, 48.0, 0.06347318278045366, 0.05813256346833801],
 [6.600137795377312e-05, 8.0, 88.0, 0.07171537179454744, 0.08131807723336026],
 [6.972502513463056e-05, 8.0, 87.0, 0.07036762292939944, 0.07697530606249184],
 [7.020840886891911e-05, 13.0, 59.0, 0.07019796738324847, 0.07644533235290442],
 [7.021387426553937e-05, 13.0, 60.0, 0.07019605585179835, 0.07643938190414415],
 [9.422030566549493e-05, 20.0, 40.0, 0.06297425113592345, 0.05696335956505656],
 [9.538483107683817e-05,
  22.0,
  43.0,
  0.06267260336149204,
  0.056267910624380094],
 [8.033683081554249e-05, 25.0, 86.0, 0.06688873626929294, 0.06680752894368305],
 [8.780281214380418e-05, 25.0, 85.0, 0.06470651771302419, 0.0611268024213488],
 [8.721455087953374e-05, 27.0, 85.0, 0.06487159470441167, 0.06153910208591807],
 [8.891766387740006e-05, 27.0, 84.0, 0.06439668177105286, 0.06036039315375293],
 [8.506221598201344e-05, 61.0, 45.0,

In [9]:
def distance_gps(gps: np.ndarray) -> np.ndarray:
    """
    Based on gps coordinates (lat, lon) calculate the distance in meters between each point, and returns the accumulated distance in meters.
    NOTE: This function is a translation of the MATLAB code from DISTANCE_GPS.m by Asmus Skar.

    Parameters
    ----------
    gps : np.ndarray
        The gps coordinates (lat, lon) in degrees

    Returns
    -------
    np.ndarray
        The accumulated distance in meters
    """
    # Extract lat and lon
    lat = gps[:, 0]
    lon = gps[:, 1]

    # Create an array for the accumulated distance
    dx = np.zeros(len(lat))

    R = 6378.137 * 1e3  # Radius of Earth in m

    # Loop through the gps coordinates and calculate the distance
    for i in range(len(dx)-1):
        dLat = np.radians(lat[i+1] - lat[i])
        dLon = np.radians(lon[i+1] - lon[i])
        a = np.sin(dLat/2)**2 + np.cos(np.radians(lat[i])) * np.cos(np.radians(lat[i+1])) * np.sin(dLon/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        dx[i+1] = dx[i] + R * c

    return dx

In [12]:
1e-4

0.0001

In [11]:
gps = np.array([[12.0001, 55.00000],[12.0001, 55.0000]])



dist = distance_gps(gps)
dist

array([ 0.        , 11.13194908])

In [ ]:
parameter_dict = {
        'acc_long':     {'bstar': 198,      'rstar': 1,     'b': 198,   'r': 0.05   },
        'acc_trans':    {'bstar': 32768,    'rstar': 1,     'b': 32768, 'r': 0.04   },
        'acc_yaw':      {'bstar': 2047,     'rstar': 1,     'b': 2047,  'r': 0.1    },
        'brk_trq_elec': {'bstar': 4096,     'rstar': -1,    'b': 4098,  'r': -1     },
        'whl_trq_est':  {'bstar': 12800,    'rstar': 0.5,   'b': 12700, 'r': 1      },
        'trac_cons':    {'bstar': 80,       'rstar': 1,     'b': 79,    'r': 1      },
        'trip_cons':    {'bstar': 0,        'rstar': 0.1,   'b': 0,     'r': 1      }
    }


def convertdata(data, parameter):
    bstar = parameter['bstar']
    rstar = parameter['rstar']
    b = parameter['b']
    r = parameter['r']
    # We only convert data in the second column at idx 1 (wrt. 0-indexing), as the first column is time
    col0 = data[:,0]
    col1 = ((data[:,1]-bstar*rstar)-b)*r
    data = np.column_stack((col0, col1))
    return data


def unpack_hdf5(hdf5_file, convert: bool = False):
    with h5py.File(hdf5_file, 'r') as f:
        print(f)
        data = unpack_hdf5_(f, convert)
    return data


def unpack_hdf5_(group, convert: bool = False):
    data = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            data[key] = unpack_hdf5_(group[key])
        else:
            if convert and key in parameter_dict:
                data[key] = convertdata(group[key][()], parameter_dict[key])
            else:
                d = group[key][()]
                if isinstance(d, bytes):
                    data[key] = d.decode('utf-8')
                else:
                    data[key] = group[key][()]
    return data


def find_best_start_and_end_indeces_by_lonlat(trip: np.ndarray, section: np.ndarray):
    # Find the start and end indeces of the section data that are closest to the trip data
    lon_a, lat_a = trip[:,0], trip[:,1]
    lon_b, lat_b = section[:,0], section[:,1]
    
    start_index = np.argmin(np.linalg.norm(np.column_stack((lon_a, lat_a)) - np.array([lon_b[0], lat_b[0]]), axis=1))
    end_index = np.argmin(np.linalg.norm(np.column_stack((lon_a, lat_a)) - np.array([lon_b[-1], lat_b[-1]]), axis=1))

    return start_index, end_index

def get_locations(p79, gm_data):
    # Extract every 10th item starting from idx[0] to idx[1]+1, to get one location for each meter
    lon_zp = p79['Lon']
    lat_zp = p79['Lat']
    idx = find_best_start_and_end_indeces_by_lonlat(p79[['Lat', 'Lon']].to_numpy(), gm_data['gps'][:,1:]) # TODO is this really the best place to start?? 
    loc_lon = lon_zp[idx[0]:idx[1]+1:10]
    loc_lat = lat_zp[idx[0]:idx[1]+1:10]

    # Combine lon and lat into a list of lists
    locations = [[lon, lat] for lon, lat in zip(loc_lon, loc_lat)]

    distance = [value for value in p79["Distance [m]"][idx[0]:idx[1]+1:10]]
    
    return locations, distance